# Analisis del comportamiento de la cuota de mercado vs distancia

Queremos predecir las ventas de una tienda basándonos en la cuota de mercado que alcanza de un CP dependiendo de la distancia a ese CP.

### Código todo en uno ejecutable

In [33]:
# Importar el modelo
import pandas as pd
import pickle
DT_model = pickle.load(open("05_Output_Regression_DT.pickle","rb"))

# Importar distancias entre CP y market size
zipcodes = pd.read_csv("03_Output_CP20km.csv")
zipcodes['Roundist'] = round (zipcodes['Dist'])

# Aplicamos el modelo para predecir el % Market share dependiendo de la distancia y lo multiplicamos por el Market Size para saber la predicción de ventas
zipcodes['MShare_Prediction'] = DT_model.predict(zipcodes[['Roundist']])
zipcodes['Sales_Prediction'] = zipcodes['Market Size'] * zipcodes['MShare_Prediction']

### Restamos de la predicción de Market Share el Market Share actual al que llegamos con nuestras tiendas, para quedarnos sólo con zonas potenciales

# Importamos el Market Share que tenemos con las tiendas actuales
actsales = pd.read_csv("04_Output_CP_Actual_MarketShare.csv")

# Calculamos los potenciales
potentials = zipcodes.merge(actsales, on='CP2', how='left').fillna(0)
potentials['Potential_Share'] = potentials['MShare_Prediction'] - potentials['Actual_MarketShare']
potentials.loc[potentials['Potential_Share'] < 0, 'Potential_Share'] = 0 
potentials['Potential_Sales'] = potentials['Market Size'] * potentials['Potential_Share']

potentials.to_csv('06_Output_Sales_Prediction.csv', sep=',',decimal=".", index=False)

--------

### Ejercicio por partes

In [1]:
# Importar el modelo
import pickle
DT_model = pickle.load(open("05_Output_Regression_DT.pickle","rb"))

In [2]:
import pandas as pd

# Importar distancias entre CP y market size
zipcodes = pd.read_csv("03_Output_CP20km.csv")
zipcodes['Roundist'] = round (zipcodes['Dist'])

# Aplicamos el modelo para predecir el % Market share dependiendo de la distancia y lo multiplicamos por el Market Size para saber la predicción de ventas
zipcodes['MShare_Prediction'] = DT_model.predict(zipcodes[['Roundist']])
zipcodes['Sales_Prediction'] = zipcodes['Market Size'] * zipcodes['MShare_Prediction']

In [3]:
zipcodes.head()

,CP,CP2,Dist,Market Size,Roundist,MShare_Prediction,Sales_Prediction
0,1001,1001,0.000000,4055236,0.0,0.100455,407368.218132
1,1001,1002,1.346059,9222645,1.0,0.084919,783182.232010
2,1001,1003,1.756450,10160240,2.0,0.084919,862802.313323
3,1001,1004,0.647104,1968115,1.0,0.084919,167131.305450
4,1001,1005,0.541742,3139321,1.0,0.084919,266589.511770


In [31]:
### Restamos de la predicción de Market Share el Market Share actual al que llegamos con nuestras tiendas, para quedarnos sólo con zonas potenciales
# Importamos el Market Share que tenemos con las tiendas actuales
actsales = pd.read_csv("04_Output_CP_Actual_MarketShare.csv")

# Calculamos los potenciales
potentials = zipcodes.merge(actsales, on='CP2', how='left').fillna(0)
potentials['Potential_Share'] = potentials['MShare_Prediction'] - potentials['Actual_MarketShare']
potentials.loc[potentials['Potential_Share'] < 0, 'Potential_Share'] = 0 
potentials['Potential_Sales'] = potentials['Market Size'] * potentials['Potential_Share']
potentials.head()

,CP,CP2,Dist,Market Size,Roundist,MShare_Prediction,Sales_Prediction,Actual_MarketShare,Potential_Share,Potential_Sales
0,1001,1001,0.000000,4055236,0.0,0.100455,407368.218132,0.010417,0.090038,365123.518132
1,1001,1002,1.346059,9222645,1.0,0.084919,783182.232010,0.000948,0.083971,774435.232010
2,1001,1003,1.756450,10160240,2.0,0.084919,862802.313323,0.000008,0.084912,862723.013323
3,1001,1004,0.647104,1968115,1.0,0.084919,167131.305450,0.010553,0.074367,146362.105450
4,1001,1005,0.541742,3139321,1.0,0.084919,266589.511770,0.004498,0.080422,252469.211770


In [32]:
Dist_sales = potentials.groupby(['CP'],as_index=False).agg({'Potential_Sales':'sum'})
Dist_sales.sort_values(by=['Potential_Sales'], ascending=False, inplace=True)
Dist_sales.head()

,CP,Potential_Sales
5399,28014,9.370943e+07
5397,28012,9.330769e+07
5392,28007,9.297092e+07
5386,28001,9.295530e+07
5394,28009,9.285825e+07
